In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="SkynetML",resource_group="ML",subscription_id="17e15d04-6670-44fd-bf24-195e65650e31")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: SkynetML
Azure region: eastus
Subscription id: 17e15d04-6670-44fd-bf24-195e65650e31
Resource group: ML


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_target = ComputeTarget(workspace=ws, name="d2-dedicated")
compute_target.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [86]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
from azureml.core import ScriptRunConfig
from azureml.core import Environment
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "C": uniform(0, 1),
        "max_iter": choice(100, 1000, 200, 300, 400, 500, 800, 900)
    })

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

sklearn_env = Environment.from_conda_specification(name="sklearn-env",file_path="conda_env.yml")
if "outputs" not in os.listdir():
    os.mkdir("./outputs")
if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = ScriptRunConfig(source_directory=".",script="train.py",compute_target=compute_target,environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=10,
                             max_concurrent_runs=1)

In [87]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

### YOUR CODE HERE ###


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [89]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n parameter values: ', parameter_values)
### YOUR CODE HERE ###

print(best_run.get_file_names())

best_run.download_file(best_run.get_file_names()[-1], output_file_path='./outputs/')


Best Run Id:  HD_b48e1f5c-1713-445c-9d01-128676200322_4

 Accuracy: 0.9186646433990895

 parameter values:  ['--C', '0.8289480633618689', '--max_iter', '500']
['azureml-logs/55_azureml-execution-tvmps_b655cd04375a3491e342b5122d3a67f15ade9141d5dda83ac9906d1c0764dafd_d.txt', 'azureml-logs/65_job_prep-tvmps_b655cd04375a3491e342b5122d3a67f15ade9141d5dda83ac9906d1c0764dafd_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_b655cd04375a3491e342b5122d3a67f15ade9141d5dda83ac9906d1c0764dafd_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/98_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/.amlignore', 'outputs/model.joblib']


In [91]:
best_run.register_model(model_name="best_hd_model", model_path='./outputs/hyperdrive-model.joblib')

Model(workspace=Workspace.create(name='SkynetML', subscription_id='17e15d04-6670-44fd-bf24-195e65650e31', resource_group='ML'), name=best_model, id=best_model:2, version=2, tags={}, properties={})

In [90]:
joblib.load('./outputs/hyperdrive-model.joblib')

Trying to unpickle estimator LogisticRegression from version 0.24.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.


LogisticRegression(C=0.8289480633618689, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=500, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [38]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = Dataset.Tabular.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")
df = ds.to_pandas_dataframe().dropna()
print(df)

       age          job  marital          education  default housing loan  \
0       57   technician  married        high.school       no      no  yes   
1       55      unknown  married            unknown  unknown     yes   no   
2       33  blue-collar  married           basic.9y       no      no   no   
3       36       admin.  married        high.school       no      no   no   
4       27    housemaid  married        high.school       no     yes   no   
...    ...          ...      ...                ...      ...     ...  ...   
32945   56    housemaid  married           basic.4y       no      no  yes   
32946   37   management  married  university.degree       no      no  yes   
32947   26       admin.   single  university.degree       no      no   no   
32948   31  blue-collar   single           basic.9y       no      no   no   
32949   39    housemaid  married           basic.4y       no      no   no   

         contact month day_of_week  ...  campaign  pdays  previous  \
0    

In [39]:
from train import clean_data
from sklearn.model_selection import train_test_split
# Use the clean_data function to clean your data.
x, y = clean_data(df)
x['y'] = y
x.to_csv('data/clean_data.csv')

print(x)



       age  marital  default  housing  loan  month  day_of_week  duration  \
0       57        1        0        0     1      5            1       371   
1       55        1        0        1     0      5            4       285   
2       33        1        0        0     0      5            5        52   
3       36        1        0        0     0      6            5       355   
4       27        1        0        1     0      7            5       189   
...    ...      ...      ...      ...   ...    ...          ...       ...   
32945   56        1        0        0     1      7            1       116   
32946   37        1        0        0     1      7            5        69   
32947   26        0        0        0     0      5            2       135   
32948   31        0        0        0     0      4            1       386   
32949   39        1        0        0     0      8            4       179   

       campaign  pdays  ...  contact_telephone  education_basic.4y  \
0    

In [41]:
datastore = ws.get_default_datastore()
datastore.upload(src_dir='data',target_path='data')


Uploading an estimated of 3 files
Uploading data/.amlignore
Uploaded data/.amlignore, 1 files out of an estimated total of 3
Uploading data/.amlignore.amltmp
Uploaded data/.amlignore.amltmp, 2 files out of an estimated total of 3
Uploading data/clean_data.csv
Uploaded data/clean_data.csv, 3 files out of an estimated total of 3
Uploaded 3 files


In [ ]:
data = Dataset.Tabular.from_delimited_files(path=[(datastore,'data/clean_data.csv')])

In [23]:
train_data, testing_data = train_test_split(data,test_size=.1)
testing_data['y'] = testing_data['y'].apply(lambda s: 1 if s =='yes' else 0)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
print(testing_data[testing_data['y'] != 0])

Empty DataFrame
Columns: [age, marital, default, housing, loan, month, day_of_week, duration, campaign, pdays, previous, poutcome, emp.var.rate, cons.price.idx, cons.conf.idx, euribor3m, nr.employed, job_admin., job_blue-collar, job_entrepreneur, job_housemaid, job_management, job_retired, job_self-employed, job_services, job_student, job_technician, job_unemployed, job_unknown, contact_cellular, contact_telephone, education_basic.4y, education_basic.6y, education_basic.9y, education_high.school, education_illiterate, education_professional.course, education_university.degree, education_unknown, y]
Index: []

[0 rows x 40 columns]


In [42]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data,
    label_column_name='y',
    validation_size=.1,
    test_size=.1,
    compute_target=compute_target)

In [43]:
# Submit your automl run
from azureml.widgets import RunDetails
### YOUR CODE HERE ###
run = exp.submit(automl_config)
RunDetails(run).show()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_c57c3bf8-ae90-4e87-9486-f3ad51910e9a,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [52]:
# Retrieve and save your best automl model.
import joblib
### YOUR CODE HERE ###
best_run, fitted_model = run.get_output()
best_run.download_file('outputs/model.pkl',output_file_path='outputs/best_automl_model.pkl')
print(fitted_model)
# joblib.dump(value=best_run,filename='./outputs/best_automl_run.joblib')

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/moun...
), random_state=0, reg_alpha=2.1875, reg_lambda=1.0416666666666667, subsample=1, tree_method='auto'))], verbose=False)), ('7', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('logisticregression', LogisticRegression(C=1.7575106248547894, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='multinomial', n_jobs=1, penalty='l2', random_state=None, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))], verbose=False)), ('4', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifie

In [53]:
joblib.load('outputs/best_automl_model.pkl')

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/moun...
), random_state=0, reg_alpha=2.1875, reg_lambda=1.0416666666666667, subsample=1, tree_method='auto'))], verbose=False)), ('7', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('logisticregression', LogisticRegression(C=1.7575106248547894, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='multinomial', n_jobs=1, penalty='l2', random_state=None, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))], verbose=False)), ('4', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifie